# 2. Download Lasair light curves

Uses the **Lasair API** to fetch light-curve data for each object in `ztf_cleansed.csv`. You choose a **run folder** name; the notebook creates `runs/<run_name>/<timestamp>_lightcurve/` and writes one CSV per object there. Original Lasair data is cached in `data/lasair/`.

Each file contains MJD, filter (g/r), and flux/errors. When forced flux is missing, **`utils.processLasairData()`** fills `forced_ujy` and `forced_ujy_error` from unforced magnitude (AB zeropoint 23.9) and then cleans and overwrites the CSV.

In [3]:
# load ztf data
import importlib
from pathlib import Path
from datetime import datetime
import pandas as pd
import lasair
import numpy as np
import shutil
import utils
importlib.reload(utils)
from utils import processLasairData, cleanLightCurve

project_root = Path.cwd().parent
data_folder = project_root / "data" / "lasair"
data_folder.mkdir(parents=True, exist_ok=True)

# create specific folder for this run
folder_name = input("Enter name for run folder (leave blank for timestamp): ").strip()
if not folder_name:
    start_time = datetime.now()
    folder_name = start_time.strftime("%Y-%m-%d_%H-%M-%S")
run_folder = project_root / "runs" / folder_name
run_folder.mkdir(parents=True, exist_ok=True)

lightcurve_dir = run_folder / "laisair"
lightcurve_dir.mkdir(parents=True, exist_ok=True)

# reade the cleansed data
ztf_data = project_root / "ztf_cleansed.csv"
ztf_df = pd.read_csv(ztf_data)
print(len(ztf_df))
print(ztf_df.head())

5010
          ZTFID      IAUID           RA          Dec     peakt peakfilt  \
0  ZTF17aabtvsy  SN2022yei  10:35:32.09  +37:38:59.0  59870.49        r   
1  ZTF17aabvong  SN2024xxq  02:05:07.68  +11:14:55.1  60606.25        g   
2  ZTF17aacldgo  SN2022zxv  03:09:24.36  -04:53:39.2  59897.25        g   
3  ZTF17aadlxmv  SN2020adv  08:29:47.59  +33:54:22.8  58879.19        g   
4  ZTF18aaaonon  SN2022jjs  10:19:05.51  +14:24:16.6  59703.27        g   

   peakmag  peakabs duration    rise      fade   type  redshift          b  \
0  18.0303   -19.41  >34.229   >6.01    28.219  SN Ia   0.06922  59.641962   
1  16.8039   -19.57   23.222   8.464    14.758  SN Ia   0.03400 -47.664064   
2  18.7979   -18.91    >1077   >3.85  >1073.15  SN Ia   0.07200 -50.332472   
3  17.9475   -19.34   25.146  10.951    14.195  SN Ia   0.06200  34.174702   
4  18.5663   -19.08  >18.631   3.691    >14.94  SN Ia   0.07141  52.363911   

     A_V  tns_redshift  
0  0.053      0.069222  
1  0.446      0.034000  


In [4]:
from typing import Any

# lasair api token
lasair_token = 'c4ce6509f05363cfb57aaedffb056ef31573e647'
client = lasair.lasair_client(lasair_token)
print("Lasair client initialized.")

num_total = len(ztf_df)
num_processed = 0
num_success = 0

for idx, row in enumerate[Any](ztf_df.itertuples(index=False)):
    num_processed += 1
    print(f"\nProcessing {num_processed:3d}/{num_total}: {row.ZTFID}...", end='')
    ztf_id = row.ZTFID
    iauid = row.IAUID
    if str(iauid).startswith('SN'):
        iauid = str(iauid)[2:]

    data_path = data_folder / f"{ztf_id}_lightcurve.csv"
    run_path = lightcurve_dir / f"{ztf_id}_lightcurve.csv"
    print(f" Checking data folder...", end='')
    # we don't need to keep downlaoding the same data. skip if already stored.
    if data_path.exists():
        print(f" Already exists.", end='')
        df_obj = pd.read_csv(data_path)
    else:
        print(f" Downloading...", end='')
        try:
            result = client.object(ztf_id)
        except Exception as e:
            print(f"\nLasair API error for {ztf_id}: {e}")
            continue

        if not result or 'candidates' not in result:
            print(f"\nNo candidates found for {ztf_id}. The object may not exist or API error.")
            continue
        if "error" in result:
            print(f"\nAPI returned error for {ztf_id}: {result.get('error')}")
            continue

        candidates = result['candidates']
        if not candidates:
            print(f"\nNo light curve data for {ztf_id}")
            continue

        output_rows = []
        for cand in candidates:
            # fid: filter ID (1=g, 2=r)
            fid = cand.get("fid")
            # keep i band for completeness             
            fid_to_letter = {1: "g", 2: "r", 3: "i"}
            filter_letter = fid_to_letter.get(fid, fid) if fid is not None else None
            
            # store light curve data as new csv
            out_row = {
                'ztf_id': ztf_id,
                'iauid': iauid,
                'MJD': cand.get('mjd', None),
                'filter': filter_letter,
                'unforced_mag': cand.get('magpsf', None),
                'unforced_mag_error': cand.get('sigmapsf', None),
                'unforced_mag_status': 'positive' if cand.get('isdiffpos', 't') == 't' else 'negative',
                'forced_ujy': cand.get('forcediffimflux', None),
                'forced_ujy_error': cand.get('forcediffimfluxunc', None),
            }
            output_rows.append(out_row)

        df_obj = pd.DataFrame(output_rows)
        df_obj.to_csv(data_path, index=False)
        print(f" Saved {len(output_rows)} rows.", end='')
    # move a copy of the lightcurve into the run folder
    print(f" Copying to run folder...", end='')
    shutil.copy(data_path, run_path)
    print(f" Copied.", end='')
    
    print(f" Processing...", end='')
    # we process the csv, ensuring they have forced_ujy and forced_ujy_error
    try:
        processLasairData(df_obj, run_path)
        num_success += 1
        print(f" Done ({num_success}/{num_processed})")
    except Exception as e:
        print(f" Error: {e} ({num_success}/{num_processed})")

    try:
        print(run_path, ztf_data)
        cleanLightCurve(run_path, ztf_data) 
    except Exception as e:
        print(f" Error in cleaning: {e}")

print(f"\nAll done! {num_success} of {num_total} objects processed successfully.")

Lasair client initialized.

Processing   1/5010: ZTF17aabtvsy... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run2/laisair/ZTF17aabtvsy_lightcurve.csv (23 rows)
 Done (1/1)
/Users/david/Code/msc/runs/run2/laisair/ZTF17aabtvsy_lightcurve.csv /Users/david/Code/msc/ztf_cleansed.csv
Cleaned light curve data in /Users/david/Code/msc/runs/run2/laisair/ZTF17aabtvsy_lightcurve.csv (22 rows)

Processing   2/5010: ZTF17aabvong... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run2/laisair/ZTF17aabvong_lightcurve.csv (31 rows)
 Done (2/2)
/Users/david/Code/msc/runs/run2/laisair/ZTF17aabvong_lightcurve.csv /Users/david/Code/msc/ztf_cleansed.csv
Cleaned light curve data in /Users/david/Code/msc/runs/run2/laisair/ZTF17aabvong_lightcurve.csv (29 rows)

Processing   3/5